# Download the Dataset and Understand the Format

In [ ]:
! pip install pydub
from pydub import AudioSegment
import keras
import IPython
import wave
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.io import wavfile
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.python.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras import layers
from keras import models
from sklearn.metrics import confusion_matrix
from tensorflow.python.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import itertools
from keras import backend as K


In [ ]:
def load_plt_audio(path):
    audio_sig,sample_rate  = librosa.load(path)#dtrunc(ata.size/sample rate) = audio length
    #plot wave plot
    plt.plot(audio_sig)#reading of pressure wave form of peaks and troughs "negative peaks"
    plt.title('Waveplot')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.show()
    #plot frequencies
    spectrum = np.fft.fft(audio_sig)# we use Fast Fourier Transform not Fourier Transform as our data is discrete 
    frequencies = np.fft.fftfreq(len(spectrum))
    magnitude = np.abs(spectrum)
    plt.plot(frequencies[:int(len(magnitude)/2)],spectrum[:int(len(magnitude)/2)])#only plot left half as right half is the same
    plt.title('Discrete-Fourier Transform')
    plt.xlabel('Frequency')
    plt.ylabel('Magnitude')
    plt.show()

    return  sample_rate, audio_sig

In [ ]:
print("neutral")
path = '../input/speech-emotion-recognition-en/Ravdess/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-01-01.wav'
sample_rate, audio_sig =load_plt_audio(path)
IPython.display.Audio(rate= sample_rate,data= audio_sig)

In [ ]:
print("calm")
path = '../input/speech-emotion-recognition-en/Ravdess/audio_speech_actors_01-24/Actor_01/03-01-02-01-01-01-01.wav'
sample_rate, audio_sig =load_plt_audio(path)
IPython.display.Audio(rate= sample_rate,data= audio_sig)

In [ ]:
print("happy")
path = '../input/speech-emotion-recognition-en/Ravdess/audio_speech_actors_01-24/Actor_01/03-01-03-01-01-01-01.wav'
sample_rate, audio_sig =load_plt_audio(path)
IPython.display.Audio(rate= sample_rate,data= audio_sig)

In [ ]:
print("sad")
path = '../input/speech-emotion-recognition-en/Ravdess/audio_speech_actors_01-24/Actor_01/03-01-04-01-01-01-01.wav'
sample_rate, audio_sig =load_plt_audio(path)
IPython.display.Audio(rate= sample_rate,data= audio_sig)

In [ ]:
print("angry")
path = '../input/speech-emotion-recognition-en/Ravdess/audio_speech_actors_01-24/Actor_01/03-01-05-01-01-01-01.wav'
sample_rate, audio_sig =load_plt_audio(path)
IPython.display.Audio(rate= sample_rate,data= audio_sig)

In [ ]:
print("fearful")
path = '../input/speech-emotion-recognition-en/Ravdess/audio_speech_actors_01-24/Actor_01/03-01-06-01-01-01-01.wav'
sample_rate, audio_sig =load_plt_audio(path)
IPython.display.Audio(rate= sample_rate,data= audio_sig)

In [ ]:
print("disgust")
path = '../input/speech-emotion-recognition-en/Ravdess/audio_speech_actors_01-24/Actor_01/03-01-07-01-01-01-01.wav'
sample_rate, audio_sig =load_plt_audio(path)
IPython.display.Audio(rate= sample_rate,data= audio_sig)

In [ ]:
print("surprised")
path = '../input/speech-emotion-recognition-en/Ravdess/audio_speech_actors_01-24/Actor_01/03-01-08-01-01-01-01.wav'
sample_rate, audio_sig =load_plt_audio(path)
IPython.display.Audio(rate= sample_rate,data= audio_sig)

load dataset

In [ ]:
emotion_ravdess = {
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprise'}

emotion_crema = {
    'FEA':'fearful',
    'NEU':'neutral',
    'ANG':'angry',
    'HAP':'happy',
    'DIS':'disgust',
    'SAD':'sad'}

emotion_savee = {
    'f':'fearful',
    'n':'neutral',
    'a':'angry',
    'h':'happy',
    'd':'disgust',
    'sa':'sad',
    'su':'surprise'}

emotion_tess = {
    'Fear':'fearful',
    'fear':'fearful',
    'neutral':'neutral',
    'angry':'angry',
    'happy':'happy',
    'disgust':'disgust',
    'sad':'sad',
    'Sad':'sad',
    'surprise':'surprise',
    'surprised':'surprise'}


In [ ]:
np_data = []

#add Ravdess data
directory = '../input/speech-emotion-recognition-en/Ravdess/audio_speech_actors_01-24'
for actor in os.scandir(directory):
    for file in os.scandir(actor.path):
        emo = emotion_ravdess[file.path.split('-')[-5]]

        np_data.append([file.path , emo])

#add crema data
directory = '../input/speech-emotion-recognition-en/Crema'
for file in os.scandir(directory):
    emo = emotion_crema[file.path.split('_')[2]]

    np_data.append([file.path , emo])

#add Tess data
directory = '../input/speech-emotion-recognition-en/Tess'
for folder in os.scandir(directory):
    emo = emotion_tess[folder.path.split('_')[-1]]
    for file in os.scandir(folder.path):
        np_data.append([file.path , emo])

#add Savee data
directory = '../input/speech-emotion-recognition-en/Savee'
for file in os.scandir(directory):
    key = file.path.split('_')[-1][0]
    if key=='s':
        key = file.path.split('_')[-1][0:2]
    emo = emotion_savee[key]

    np_data.append([file.path , emo])

df = pd.DataFrame(np_data, columns = ['Path', 'Emotion'])


In [ ]:
df['Emotion'].value_counts()

In [ ]:
# df2 = df.copy()

df = df[df.Emotion != "surprise"]
df = df[df.Emotion != "calm"]

In [ ]:
df['Emotion'].value_counts()

In [ ]:
df.reset_index(drop=True, inplace=True)

# Create the Feature Space

In [ ]:
def load_audio(path):
    sig, sample_rate = librosa.load(path,offset=0.3)
    new_siganl = np.zeros(64745)
    new_siganl[:len(sig)] =  sig[:64745]        
    return new_siganl,sample_rate

features domain

In [ ]:
def noise(data, random=False, rate=0.035, threshold=0.075):
    if random:
        rate = np.random.random() * threshold
    noise_amp = rate*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data


def pitch(data, sampling_rate, pitch_factor=0.7, random=False):#make it lower or higher
    if random:
        pitch_factor=np.random.random() * pitch_factor
    return librosa.effects.pitch_shift(data, sampling_rate, (int)(pitch_factor))

In [ ]:
# def zcr(data, frame_length=2048, hop_length=512):
#     zcr = librosa.feature.zero_crossing_rate(y=data, frame_length=frame_length, hop_length=hop_length)
#     return np.squeeze(zcr)


# def energy(data, frame_length=2048, hop_length=512):
#     en = np.array([np.sum(np.power(np.abs(data[hop:hop+frame_length]), 2)) for hop in range(0, data.shape[0], hop_length)])
#     return en / frame_length


# def rmse(data, frame_length=2048, hop_length=512):
#     rmse = librosa.feature.rms(y=data, frame_length=frame_length, hop_length=hop_length)
#     return np.squeeze(rmse)


# def entropy_of_energy(data, frame_length=2048, hop_length=512):
#     energies = energy(data, frame_length, hop_length)
#     energies /= np.sum(energies)

#     entropy = 0.0
#     entropy -= energies * np.log2(energies)
#     return entropy


# def spc(data, sr, frame_length=2048, hop_length=512):
#     spectral_centroid = librosa.feature.spectral_centroid(y=data, sr=sr, n_fft=frame_length, hop_length=hop_length)
#     return np.squeeze(spectral_centroid)


# def spc_flux(data):
#     isSpectrum = data.ndim == 1
#     if isSpectrum:
#         data = np.expand_dims(data, axis=1)
#     X = np.c_[data[:, 0], data]
#     af_Delta_X = np.diff(X, 1, axis=1)
#     vsf = np.sqrt((np.power(af_Delta_X, 2).sum(axis=0))) / X.shape[0]

#     return np.squeeze(vsf) if isSpectrum else vsf


# def spc_rollof(data, sr, frame_length=2048, hop_length=512):
#     spcrollof = librosa.feature.spectral_rolloff(y=data, sr=sr, n_fft=frame_length, hop_length=hop_length)
#     return np.squeeze(spcrollof)

# def mfcc(data, sr, frame_length=2048, hop_length=512, flatten: bool = True):
#     mfcc_feature = librosa.feature.mfcc(y=data, sr=sr)
#     return np.squeeze(mfcc_feature.T) if not flatten else np.ravel(mfcc_feature.T)



In [ ]:
# def extract_features(data, sr, frame_length=2048, hop_length=512):
#     result = np.array([])
#     result = np.hstack((result,
#                         zcr(data, frame_length, hop_length),
# #                         np.mean(energy(data, frame_length, hop_length),axis=0),
# #                         np.mean(entropy_of_energy(data, frame_length, hop_length), axis=0),
#                         rmse(data, frame_length, hop_length),
# #                         spc(data, sr, frame_length, hop_length),
# #                         spc_flux(data),
# #                         spc_rollof(data, sr, frame_length, hop_length),
#                          mfcc(data, sr, frame_length, hop_length)

#                                     ))
#     return result

In [ ]:
# X, Y = [], []


# for i in df.index:
#     sig, sample_rate  = load_audio(df["Path"][i])
#     features = extract_features(sig,sample_rate)
#     X.append(features)
#     Y.append( df['Emotion'][i])
    
#     noise_data = noise(sig, random=True)
#     features2 = extract_features(noise_data, sample_rate)
#     X.append(features2)
#     Y.append( df['Emotion'][i])
    
#     # data with pitching
#     pitched_data = pitch(sig, sample_rate, random=True)
#     features3 = extract_features(pitched_data, sample_rate)
#     X.append(features3)
#     Y.append( df['Emotion'][i])
    
#     # data with pitching and white_noise
#     new_data = pitch(sig, sample_rate, random=True)
#     data_noise_pitch = noise(new_data, random=True)
#     features4 = extract_features(data_noise_pitch, sample_rate)
#     X.append(features4)
#     Y.append( df['Emotion'][i])
#     if i%100 == 0:
#         print(i)
# df_features = pd.DataFrame(X)
# df_features["Emotion"] = Y

In [ ]:
# df_features = df_features.fillna(0)
# print(df_features.isna().any().sum())

In [ ]:
# df_features

mel spectrogram

In [ ]:
path = '../input/speech-emotion-recognition-en/Tess/YAF_fear/YAF_back_fear.wav'
samplerate, audio_sig =load_plt_audio(path)

#Compute a mel-scaled spectrogram.
mel_signal = librosa.feature.melspectrogram(y=audio_sig, sr=samplerate)
# gathering the absolute values for all values in our audio_stft 
spectrogram = np.abs(mel_signal)
# Converting the amplitude to decibels
power_to_db = librosa.power_to_db(spectrogram, ref=np.max)


librosa.display.specshow(power_to_db, sr=samplerate, x_axis='time', y_axis='mel', cmap='magma')
plt.colorbar(label='dB')
plt.title('Mel-Spectrogram (dB)')
plt.xlabel('Time')
plt.ylabel('Mel Scale')
plt.show()

In [ ]:
X, Y = [], []


for i in df.index:
    sig, sample_rate  = load_audio(df["Path"][i])
    mel_signal = librosa.feature.melspectrogram(y=sig, sr=sample_rate)
    spectrogram = np.abs(mel_signal)
    power_to_db = librosa.power_to_db(spectrogram, ref=np.max)
    X.append(power_to_db)
    Y.append( df['Emotion'][i])

    # data with noise
    noise_data = noise(sig, random=True)
    mel_signal = librosa.feature.melspectrogram(y=noise_data, sr=sample_rate)
    spectrogram = np.abs(mel_signal)
    power_to_db = librosa.power_to_db(spectrogram, ref=np.max)
    X.append(power_to_db)
    Y.append( df['Emotion'][i])
    
    # data with pitching
    pitched_data = pitch(sig, sample_rate, random=True)
    mel_signal = librosa.feature.melspectrogram(y=pitched_data, sr=sample_rate)
    spectrogram = np.abs(mel_signal)
    power_to_db = librosa.power_to_db(spectrogram, ref=np.max)
    X.append(power_to_db)
    Y.append( df['Emotion'][i])
    
    #data with pitching and white_noise
    new_data = pitch(sig, sample_rate, random=True)
    data_noise_pitch = noise(new_data, random=True)
    mel_signal = librosa.feature.melspectrogram(y=data_noise_pitch, sr=sample_rate)
    spectrogram = np.abs(mel_signal)
    power_to_db = librosa.power_to_db(spectrogram, ref=np.max)
    X.append(power_to_db)
    Y.append( df['Emotion'][i])

    if i%100 ==0:
        print(i)

X = np.array(X)
Y = np.array(Y)
shape = X.shape
X = X.reshape((X.shape[0],-1))
# df_features = pd.DataFrame(X)
# df_features["Emotion"] = Y

In [ ]:
# print(df_features.isna().any().sum())
# df_features = df_features.fillna(0)
# print(df_features.isna().any().sum())

# Building the Model

1d - model

In [ ]:
# X = df_features.drop(labels="Emotion", axis=1)
# Y = df_features["Emotion"]
# Y

In [ ]:
lb = LabelEncoder()
Y = np_utils.to_categorical(lb.fit_transform(Y))
print(lb.classes_)
Y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.3, shuffle=True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42, test_size=0.05, shuffle=True)
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape

In [ ]:
X_train

In [ ]:
# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape

In [ ]:
# # We have to use 2-dimensional CNN which need specifical shape:
X_train = X_train.reshape((-1,shape[1],shape[2]))
X_test = X_test.reshape((-1,shape[1],shape[2]))
X_val = X_val.reshape((-1,shape[1],shape[2]))

X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)
X_val = np.expand_dims(X_val, axis=3)

X_train.shape, X_test.shape, X_val.shape

In [ ]:
# # We have to use 1-dimensional CNN which need specifical shape:
# X_train = np.expand_dims(X_train, axis=2)
# X_val = np.expand_dims(X_val, axis=2)
# X_test = np.expand_dims(X_test, axis=2)
# X_train.shape

In [ ]:
def recall_cal(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_cal(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f_score(y_true, y_pred):
    precision = recall_cal(y_true, y_pred)
    recall = precision_cal(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# def Res1D(layer_in, n_filters):
#     x_skip = layer_in
#     x = layers.Conv1D(n_filters, kernel_size=3, strides=1,padding="same")(layer_in)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("LeakyReLU")(x)
#     x = layers.Conv1D(n_filters, kernel_size=3, strides=1,padding="same")(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Add()([x, x_skip])
#     x = layers.Activation("LeakyReLU")(x)
#     return x

In [ ]:
X=0
Y=0

In [ ]:
 def Res2D(layer_in, n_filters):
    x_skip = layer_in
    x = layers.Conv2D(n_filters, kernel_size=(3,3), strides=(1,1),padding="same")(layer_in)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("LeakyReLU")(x)
    x = layers.Conv2D(n_filters, kernel_size=(3,3), strides=(1,1),padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, x_skip])
    x = layers.Activation("LeakyReLU")(x)
    return x

In [ ]:
model_input = keras.Input((X_train.shape[1],X_train.shape[2],1),  name="audio")

x = layers.Conv2D(128, kernel_size=(3,3), strides=(3,3),padding="same")(model_input)
x = Res2D(x,128)
x = layers.MaxPool2D(pool_size=(3,3), strides=(3,3), padding="same")(x)

x = Res2D(x,128)
x = layers.MaxPool2D(pool_size=(3,3), strides=(3,3), padding="same")(x)

x = layers.Conv2D(256, kernel_size=(3,3), strides=(1,1),padding="same")(x)
x = Res2D(x,256)
x = layers.MaxPool2D(pool_size=(3,3), strides=(3,3), padding="same")(x)

x = Res2D(x,256)
x = layers.MaxPool2D(pool_size=(3,3), strides=(3,3), padding="same")(x)

x = Res2D(x,256)
x = layers.MaxPool2D(pool_size=(3,3), strides=(3,3), padding="same")(x)

x = Res2D(x,256)
x = layers.MaxPool2D(pool_size=(3,3), strides=(3,3), padding="same")(x)


x = Res2D(x,256)
x = layers.MaxPool2D(pool_size=(3,3), strides=(3,3), padding="same")(x)


x = layers.Conv2D(512, kernel_size=(3,3), strides=(3,3),padding="same")(x)
x = Res2D(x,512)
x = layers.MaxPool2D(pool_size=(3,3), strides=(3,3), padding="same")(x)
x = layers.Conv2D(512, kernel_size=(3,3), strides=(1,1),padding="same")(x)

x = layers.Flatten()(x)
model_output = layers.Dense(6, activation="softmax")(x)


model = keras.Model(model_input, model_output, name="2D-CNN")
model.summary()
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc", f_score])

In [ ]:
learningratereduction = ReduceLROnPlateau(monitor='val_acc', patience=3,verbose=1, factor=0.5, min_lr=0.00001)
earlystopping = EarlyStopping(monitor ="val_acc", patience = 5, restore_best_weights = True)


history=model.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_val, y_val),callbacks=[earlystopping,learningratereduction])

In [ ]:
print("Accuracy of our model on test data : " , model.evaluate(X_test,y_test)[1]*100 , "%")

fig , ax = plt.subplots(1,2)
train_acc = history.history['acc']
train_loss = history.history['loss']
test_acc = history.history['val_acc']
test_loss = history.history['val_loss']

fig.set_size_inches(20,6)
ax[0].plot(train_loss, label = 'Training Loss')
ax[0].plot(test_loss , label = 'Testing Loss')
ax[0].set_title('Training & Testing Loss')
ax[0].legend()
ax[0].set_xlabel("Epochs")

ax[1].plot(train_acc, label = 'Training Accuracy')
ax[1].plot(test_acc , label = 'Testing Accuracy')
ax[1].set_title('Training & Testing Accuracy')
ax[1].legend()
ax[1].set_xlabel("Epochs")
plt.show()

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_pred
y_check = np.argmax(y_test, axis=1)
y_check
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true=y_check, y_pred=y_pred)

In [ ]:

def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
cm_plot_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
# model_input = keras.Input((X_train.shape[1],1),  name="audio")

# x = layers.Conv1D(128, kernel_size=3, strides=3,padding="same")(model_input)
# x = Res1D(x,128)
# x = layers.MaxPool1D(pool_size=3, strides=3, padding="same")(x)

# x = Res1D(x,128)
# x = layers.MaxPool1D(pool_size=3, strides=3, padding="same")(x)

# x = layers.Conv1D(256, kernel_size=3, strides=1,padding="same")(x)
# x = Res1D(x,256)
# x = layers.MaxPool1D(pool_size=3, strides=3, padding="same")(x)

# x = Res1D(x,256)
# x = layers.MaxPool1D(pool_size=3, strides=3, padding="same")(x)

# x = Res1D(x,256)
# x = layers.MaxPool1D(pool_size=3, strides=3, padding="same")(x)

# x = Res1D(x,256)
# x = layers.MaxPool1D(pool_size=3, strides=3, padding="same")(x)


# x = Res1D(x,256)
# x = layers.MaxPool1D(pool_size=3, strides=3, padding="same")(x)


# x = layers.Conv1D(512, kernel_size=3, strides=1,padding="same")(x)
# x = Res1D(x,512)
# x = layers.MaxPool1D(pool_size=3, strides=3, padding="same")(x)
# x = layers.Conv1D(512, kernel_size=3, strides=1,padding="same")(x)

# x = layers.Flatten()(x)
# model_output = layers.Dense(6, activation="softmax")(x)


# model = keras.Model(model_input, model_output, name="1D-CNN")
# model.summary()
# model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc", f_score])

In [ ]:
# df_mel = []

# for i in df.index:#change to df
#     mel_signal = librosa.feature.melspectrogram(y=df['Signal'][i], sr=df['Sample Rate'][i])
#     spectrogram = np.abs(mel_signal)
#     power_to_db = librosa.power_to_db(spectrogram, ref=np.max)
#     df_mel.append([power_to_db , df['Emotion'][i]]) 
    
# df_mel = pd.DataFrame(df_mel, columns = ['mel spectrogram', 'Emotion'])
# # for i in df_2d['mel spectrogram']:
# #     print(i.shape)